In [496]:
import pandas as pd
from pivottablejs import pivot_ui
from sklearn.feature_extraction.text import HashingVectorizer

In [497]:
data_frame = pd.read_csv('../data/data.csv')
pivot_ui(data_frame)
print(data_frame.shape)

(5508, 23)


In [498]:
data_frame.head()

,Unnamed: 0,MATCH_DATE,LEAGUE_NAME,SEASON,LEAGUE,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,...,HOME_ICON,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY
0,0,2005-08-05 20:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,München,NaN,1. Spieltag,1,...,https://i.imgur.com/jJEsJrj.png,87,Borussia Mönchengladbach,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0
1,1,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Köln,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/en/thum...,81,1. FSV Mainz 05,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0
2,2,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Duisburg,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/en/c/c8...,16,VfB Stuttgart,Stuttgart,https://i.imgur.com/v0tkpNx.png,1,1,1.0,0.0,0.0
3,3,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Hamburg,NaN,1. Spieltag,1,...,https://upload.wikimedia.org/wikipedia/commons...,79,1. FC Nürnberg,Nürnberg,https://upload.wikimedia.org/wikipedia/commons...,3,0,0.0,1.0,0.0
4,4,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Wolfsburg,NaN,1. Spieltag,1,...,https://i.imgur.com/ucqKV4B.png,7,Borussia Dortmund,BVB,https://upload.wikimedia.org/wikipedia/commons...,2,2,1.0,0.0,0.0


In [499]:
data_frame.columns

Index(['Unnamed: 0', 'MATCH_DATE', 'LEAGUE_NAME', 'SEASON', 'LEAGUE',
       'FINISHED', 'LOCATION', 'VIEWER', 'MATCHDAY', 'MATCHDAY_NR',
       'HOME_TEAM_ID', 'HOME_TEAM_NAME', 'HOME_TEAM', 'HOME_ICON',
       'AWAY_TEAM_ID', 'AWAY_TEAM_NAME', 'AWAY_TEAM', 'AWAY_ICON',
       'GOALS_HOME', 'GOALS_AWAY', 'DRAW', 'WIN_HOME', 'WIN_AWAY'],
      dtype='object')

In [542]:
data_frame['HOME_TEAM_ID'].value_counts().keys().shape[0]

36

In [500]:
data_frame[['HOME_TEAM', 'HOME_TEAM_NAME']]

,HOME_TEAM,HOME_TEAM_NAME
0,Bayern,FC Bayern München
1,Köln,1. FC Köln
2,Duisburg,MSV Duisburg
3,HSV,Hamburger SV
4,Wolfsburg,VfL Wolfsburg
...,...,...
5503,Köln,1. FC Köln
5504,Leipzig,RB Leipzig
5505,Stuttgart,VfB Stuttgart
5506,Union Berlin,1. FC Union Berlin


In [501]:
df = data_frame.copy()

In [502]:
df.set_index('Unnamed: 0', inplace=True)

In [503]:
df['MATCH_DATE'] = pd.to_datetime(df['MATCH_DATE'])

In [504]:
df[['MATCH_DATE', 'WIN_HOME', 'WIN_AWAY']].corr()

,MATCH_DATE,WIN_HOME,WIN_AWAY
MATCH_DATE,1.000000,-0.028222,-0.007423
WIN_HOME,-0.028222,1.000000,-0.541731
WIN_AWAY,-0.007423,-0.541731,1.000000


In [505]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5508 entries, 0 to 5509
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   MATCH_DATE      5508 non-null   datetime64[ns]
 1   LEAGUE_NAME     5508 non-null   object        
 2   SEASON          5508 non-null   int64         
 3   LEAGUE          5508 non-null   object        
 4   FINISHED        5508 non-null   bool          
 5   LOCATION        5508 non-null   object        
 6   VIEWER          1682 non-null   float64       
 7   MATCHDAY        5508 non-null   object        
 8   MATCHDAY_NR     5508 non-null   int64         
 9   HOME_TEAM_ID    5508 non-null   int64         
 10  HOME_TEAM_NAME  5508 non-null   object        
 11  HOME_TEAM       5508 non-null   object        
 12  HOME_ICON       5508 non-null   object        
 13  AWAY_TEAM_ID    5508 non-null   int64         
 14  AWAY_TEAM_NAME  5508 non-null   object        
 15  AWAY_TEAM

In [506]:
df['MATCH_DATE'].dt.month_name().value_counts()

MATCH_DATE
April        683
February     669
March        596
September    592
October      583
November     577
December     525
May          473
August       413
January      350
June          47
Name: count, dtype: int64

In [507]:
def set_season_year(month_name: str) -> str:
    month_name = pd.to_datetime(month_name).month_name()
    match month_name:
        case 'December' | 'January' | 'February':
            return 'winter'
        case 'March' | 'April' | 'May':
            return 'spring'
        case 'June' | 'July' | 'August':
            return 'summer'
        case 'September' | 'November' | 'December':
            return 'autumn'

In [508]:
df['season_year'] = df['MATCH_DATE'].apply(set_season_year)

In [509]:
df['season_year'].value_counts()

season_year
spring    1752
winter    1544
autumn    1169
summer     460
Name: count, dtype: int64

In [510]:
df['MATCH_DATE'].dt.weekday.value_counts()

MATCH_DATE
5    3521
6    1143
4     476
2     192
1     146
0      26
3       4
Name: count, dtype: int64

In [511]:
df['is_week_end'] = df['MATCH_DATE'].dt.weekday.apply(lambda x: True if x == 0 or x == 6 else False)

In [512]:
df['is_week_end'].value_counts()

is_week_end
False    4339
True     1169
Name: count, dtype: int64

In [513]:
df['MATCH_DATE'].dt.hour.value_counts()

MATCH_DATE
15    3531
20     811
18     563
17     558
19      24
13      20
21       1
Name: count, dtype: int64

In [514]:
def set_part_of_the_day(h: int) -> str:
    if h >= 0 and h < 12:
        return 'morning'
    elif h >= 12 and h < 18:
        return 'afternoon'
    else:
        return 'evening'

In [515]:
df['part_of_the_day'] = df['MATCH_DATE'].dt.hour.apply(set_part_of_the_day)

In [516]:
df['part_of_the_day'].value_counts()

part_of_the_day
afternoon    4109
evening      1399
Name: count, dtype: int64

In [517]:
df.drop('MATCH_DATE', axis=1, inplace=True)

In [518]:
df.head()

,LEAGUE_NAME,SEASON,LEAGUE,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,HOME_TEAM_ID,HOME_TEAM_NAME,...,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY,season_year,is_week_end,part_of_the_day
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,München,NaN,1. Spieltag,1,40,FC Bayern München,...,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0,summer,False,evening
1,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Köln,NaN,1. Spieltag,1,65,1. FC Köln,...,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0,summer,False,afternoon
2,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Duisburg,NaN,1. Spieltag,1,107,MSV Duisburg,...,Stuttgart,https://i.imgur.com/v0tkpNx.png,1,1,1.0,0.0,0.0,summer,False,afternoon
3,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Hamburg,NaN,1. Spieltag,1,100,Hamburger SV,...,Nürnberg,https://upload.wikimedia.org/wikipedia/commons...,3,0,0.0,1.0,0.0,summer,False,afternoon
4,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Wolfsburg,NaN,1. Spieltag,1,131,VfL Wolfsburg,...,BVB,https://upload.wikimedia.org/wikipedia/commons...,2,2,1.0,0.0,0.0,summer,False,afternoon


In [519]:
df['LEAGUE_NAME'].value_counts()

LEAGUE_NAME
1. Fussball-Bundesliga 2005/2006    306
1. Fussball-Bundesliga 2006/2007    306
1. Fußball-Bundesliga 2021/2022     306
1. Fußball-Bundesliga 2020/2021     306
1. Fußball-Bundesliga 2019/2020     306
1. Fußball-Bundesliga 2018/2019     306
1. Fußball-Bundesliga 2017/2018     306
1. Fußball-Bundesliga 2016/2017     306
1. Fußball-Bundesliga 2015/2016     306
1. Fußball-Bundesliga 2014/2015     306
1. Fussball-Bundesliga 2013/2014    306
1. Fussball-Bundesliga 2012/2013    306
1. Fussball-Bundesliga 2011/2012    306
1. Fussball-Bundesliga 2010/2011    306
1. Fussball-Bundesliga 2009/2010    306
1. Fussball-Bundesliga 2008/2009    306
1. Fußball-Bundesliga 2007/2008     306
1. Fußball-Bundesliga 2022/2023     306
Name: count, dtype: int64

In [520]:
df['SEASON'].value_counts()

SEASON
2005    306
2006    306
2021    306
2020    306
2019    306
2018    306
2017    306
2016    306
2015    306
2014    306
2013    306
2012    306
2011    306
2010    306
2009    306
2008    306
2007    306
2022    306
Name: count, dtype: int64

In [521]:
df

,LEAGUE_NAME,SEASON,LEAGUE,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,HOME_TEAM_ID,HOME_TEAM_NAME,...,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY,season_year,is_week_end,part_of_the_day
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,München,NaN,1. Spieltag,1,40,FC Bayern München,...,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0,summer,False,evening
1,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Köln,NaN,1. Spieltag,1,65,1. FC Köln,...,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0,summer,False,afternoon
2,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Duisburg,NaN,1. Spieltag,1,107,MSV Duisburg,...,Stuttgart,https://i.imgur.com/v0tkpNx.png,1,1,1.0,0.0,0.0,summer,False,afternoon
3,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Hamburg,NaN,1. Spieltag,1,100,Hamburger SV,...,Nürnberg,https://upload.wikimedia.org/wikipedia/commons...,3,0,0.0,1.0,0.0,summer,False,afternoon
4,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Wolfsburg,NaN,1. Spieltag,1,131,VfL Wolfsburg,...,BVB,https://upload.wikimedia.org/wikipedia/commons...,2,2,1.0,0.0,0.0,summer,False,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5505,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Köln,NaN,34. Spieltag,34,65,1. FC Köln,...,Bayern,https://i.imgur.com/jJEsJrj.png,1,2,0.0,0.0,1.0,spring,False,afternoon
5506,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Leipzig,NaN,34. Spieltag,34,1635,RB Leipzig,...,Schalke,https://upload.wikimedia.org/wikipedia/commons...,4,2,0.0,1.0,0.0,spring,False,afternoon
5507,1. Fußball-Bundesliga 2022/2023,2022,bl1,True,Stuttgart,NaN,34. Spieltag,34,16,VfB Stuttgart,...,Hoffenheim,https://i.imgur.com/gF0PfEl.png,1,1,1.0,0.0,0.0,spring,False,afternoon


In [522]:
df.drop(['LEAGUE_NAME', 'LEAGUE', 'FINISHED', 'VIEWER'], axis=1, inplace=True)

In [523]:
df['LOCATION'].value_counts().keys()

Index(['Berlin', 'München', 'Wolfsburg', 'Dortmund', 'Leverkusen', 'Gladbach',
       'Bremen', 'Gelsenkirchen', 'Frankfurt', 'Mainz', 'Stuttgart',
       'Sinsheim', 'Hamburg', 'Hannover', 'Freiburg', 'Köln', 'Augsburg',
       'Nürnberg', 'Leipzig', 'Bochum', 'Bielefeld', 'Cottbus',
       'Kaiserslautern', 'Düsseldorf', 'Karlsruhe', 'Duisburg', 'Fürth',
       'Paderborn', 'Darmstadt', 'Ingolstadt', 'Aachen', 'Braunschweig',
       'Rostock'],
      dtype='object', name='LOCATION')

In [524]:
from sklearn.feature_extraction.text import HashingVectorizer

In [525]:
vectorizer = HashingVectorizer(n_features=15, binary=True)
vectorizer.fit(df['LOCATION'].value_counts().keys())

HashingVectorizer(binary=True, n_features=15)

In [526]:
vectorizer.transform(['Rostock']).toarray()[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [527]:
location_vect = df['LOCATION'].apply(lambda x: vectorizer.transform([x]).toarray()[0])

In [528]:
df['match_day_num'] = df['MATCHDAY'].apply(lambda x: int(x.split('.')[0]))

In [529]:
df.drop('MATCHDAY', inplace=True, axis=1)

In [530]:
df.drop(['HOME_TEAM_NAME', 'HOME_ICON', 'HOME_TEAM', 'AWAY_TEAM_NAME', 'AWAY_ICON', 'AWAY_TEAM'], axis=1, inplace=True)

In [531]:
df_mirror = df.copy()

In [532]:
df['home_game'] = True
df_mirror['home_game'] = False

In [533]:
df.rename(columns={
    'HOME_TEAM_ID': 'TEAM_ID', 
    'AWAY_TEAM_ID': 'RIVAl_ID',
    'WIN_HOME': 'IS_WIN'}, inplace=True)
df.drop(['GOALS_AWAY', 'GOALS_HOME', 'WIN_AWAY'], axis=1, inplace=True)
df_mirror.rename(columns={
    'HOME_TEAM_ID': 'RIVAl_ID', 
    'AWAY_TEAM_ID': 'TEAM_ID',
    'WIN_AWAY': 'IS_WIN'}, inplace=True)
df_mirror.drop(['GOALS_AWAY', 'GOALS_HOME', 'WIN_HOME'], axis=1, inplace=True)

In [534]:
df

,SEASON,LOCATION,MATCHDAY_NR,TEAM_ID,RIVAl_ID,DRAW,IS_WIN,season_year,is_week_end,part_of_the_day,match_day_num,home_game
Unnamed: 0,,,,,,,,,,,,
0,2005,München,1,40,87,0.0,1.0,summer,False,evening,1,True
1,2005,Köln,1,65,81,0.0,1.0,summer,False,afternoon,1,True
2,2005,Duisburg,1,107,16,1.0,0.0,summer,False,afternoon,1,True
3,2005,Hamburg,1,100,79,0.0,1.0,summer,False,afternoon,1,True
4,2005,Wolfsburg,1,131,7,1.0,0.0,summer,False,afternoon,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5505,2022,Köln,34,65,40,0.0,0.0,spring,False,afternoon,34,True
5506,2022,Leipzig,34,1635,9,0.0,1.0,spring,False,afternoon,34,True
5507,2022,Stuttgart,34,16,175,1.0,0.0,spring,False,afternoon,34,True


In [535]:
res_df = pd.concat([df, df_mirror])

In [536]:
res_df

,SEASON,LOCATION,MATCHDAY_NR,TEAM_ID,RIVAl_ID,DRAW,IS_WIN,season_year,is_week_end,part_of_the_day,match_day_num,home_game
Unnamed: 0,,,,,,,,,,,,
0,2005,München,1,40,87,0.0,1.0,summer,False,evening,1,True
1,2005,Köln,1,65,81,0.0,1.0,summer,False,afternoon,1,True
2,2005,Duisburg,1,107,16,1.0,0.0,summer,False,afternoon,1,True
3,2005,Hamburg,1,100,79,0.0,1.0,summer,False,afternoon,1,True
4,2005,Wolfsburg,1,131,7,1.0,0.0,summer,False,afternoon,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5505,2022,Köln,34,40,65,0.0,1.0,spring,False,afternoon,34,False
5506,2022,Leipzig,34,9,1635,0.0,0.0,spring,False,afternoon,34,False
5507,2022,Stuttgart,34,175,16,1.0,0.0,spring,False,afternoon,34,False


In [537]:
res_df['LOCATION'].value_counts().shape

(33,)